In [1]:
import pandas as pd # standard python data library
import geopandas as gp # the geo-version of pandas
import numpy as np 
import os
import fiona
from statistics import mean, median
from pandas import read_csv
gp.io.file.fiona.drvsupport.supported_drivers['KML'] = 'rw' #To load KML files
import string
import xml.etree.ElementTree as et

# Alaska

## Import VEST File

In [2]:
vest_ak_16 = gp.read_file("./raw-from-source/VEST/ak_2016/ak_2016.shp")

## VEST Documentation

### Data Sources

> - Election results from Alaska Division of Elections (http://www.elections.alaska.gov/results/16GENR/index.shtml)
> - Precinct shapefile from Alaska Division of Elections (http://www.elections.alaska.gov/Core/districtmaps.php)

### Processing

> Early, Absentee, and Questioned votes are only reported at the State House district level (since Alaska has portions of the state with no lower level of government, HDs serve a similar purpose as counties do in other states for the purpose of reporting votes). These votes are apportioned to precincts by candidate in the same shares that the Election Day vote was split among precincts within an HD.

### Races 

> G16PRERTRU - Donald J. Trump (Republican Party)  
G16PREDCLI - Hillary Clinton (Democratic Party)  
G16PRELJOH - Gary Johnson (Libertarian Party)  
G16PREGSTE - Jill Stein (Green Party)  
G16PRECCAS - Darrell L. Castle (Constitution Party)  
G16PREIFUE - Roque De La Fuente (Independent)  
G16PREOWRI - Write-in Votes    

> G16USSRMUR - Lisa Murkowski (Republican Party)  
G16USSDMET - Ray Metcalfe (Democratic Party)  
G16USSLMIL - Joe Miller (Libertarian Party)  
G16USSICRA - Breck A. Craig (Independent)  
G16USSIGIA - Ted Gianoutsos (Independent)  
G16USSISTO - Margaret Stock (Independent)  
G16USSOWRI - Write-in Votes    
  
> G16HALRYOU - Don Young (Republican Party)  
G16HALDLIN - Steve Lindbeck (Democratic Party)  
G16HALLMCD - Jim C. McDermott (Libertarian Party)  
G16HALISOU - Bernie Souphanavong (Independent)  
G16HALOWRI - Write-in Votes 

## Election Results

### Load and Clean Election Results File

In [3]:
ak_2016 = pd.read_csv("./raw-from-source/Election_Results/resultsbyprct.txt",header=None)
print(ak_2016.head(10))

                0                 1                               2     3  \
0  01-446 Aurora   Race Statistics             Number of Precincts    NP    
1  01-446 Aurora   Race Statistics   Number of Precincts Reporting    NP    
2  01-446 Aurora      US PRESIDENT    Number of Precincts for Race    NP    
3  01-446 Aurora      US PRESIDENT   Number of Precincts Reporting    NP    
4  01-446 Aurora      US PRESIDENT               Registered Voters    NP    
5  01-446 Aurora      US PRESIDENT                   Times Counted    NP    
6  01-446 Aurora      US PRESIDENT              Castle, Darrell L.   CON    
7  01-446 Aurora      US PRESIDENT                Clinton, Hillary   DEM    
8  01-446 Aurora      US PRESIDENT             De La Fuente, Roque    NA    
9  01-446 Aurora      US PRESIDENT                   Johnson, Gary   LIB    

        4     5   6  
0  Total      1 NaN  
1  Total      1 NaN  
2  Total      1 NaN  
3  Total      1 NaN  
4  Total   2486 NaN  
5  Total    874 NaN 

#### Create a column to pivot on (this will help us keep track of the different write ins)

In [4]:
ak_2016["race_cand"]=ak_2016[1]+"-"+ak_2016[2]

#### Drop unused columns and filter down to relevant races

In [5]:
ak_2016.drop(columns=[4,6],inplace=True)

race_list = ['US PRESIDENT ','US SENATOR ','US REPRESENTATIVE ']
ak_2016 = ak_2016[ak_2016[1].isin(race_list)]

#### Filter out relevant counts (only interested in candidates)

In [6]:
filter_out_cands = ['Number of Precincts for Race ','Number of Precincts Reporting ','Registered Voters ','Times Counted ']

ak_2016 = ak_2016[~ak_2016[2].isin(filter_out_cands)]

### Pivot the data

In [7]:
ak_2016_pvt = pd.pivot_table(ak_2016,index=[0],values=[5],columns=["race_cand"],aggfunc=sum)

### Clean the columns

In [8]:
#### Clean up multi index

In [9]:
print(ak_2016_pvt.columns)
ak_2016_pvt.columns=ak_2016_pvt.columns.droplevel(0)

MultiIndex([(5,        'US PRESIDENT -Castle, Darrell L. '),
            (5,          'US PRESIDENT -Clinton, Hillary '),
            (5,       'US PRESIDENT -De La Fuente, Roque '),
            (5,             'US PRESIDENT -Johnson, Gary '),
            (5,               'US PRESIDENT -Stein, Jill '),
            (5,          'US PRESIDENT -Trump, Donald J. '),
            (5,               'US PRESIDENT -Write-in 60 '),
            (5,      'US REPRESENTATIVE -Lindbeck, Steve '),
            (5,   'US REPRESENTATIVE -McDermott, Jim C.  '),
            (5, 'US REPRESENTATIVE -Souphanavong, Bernie '),
            (5,          'US REPRESENTATIVE -Write-in 50 '),
            (5,           'US REPRESENTATIVE -Young, Don '),
            (5,            'US SENATOR -Craig, Breck A.  '),
            (5,             'US SENATOR -Gianoutsos, Ted '),
            (5,               'US SENATOR -Metcalfe, Ray '),
            (5,                 'US SENATOR -Miller, Joe '),
            (5,         

#### Rename and reorder the columns to match VEST

In [10]:
print(ak_2016_pvt.columns)

data_columns = ['G16PRECCAS','G16PREDCLI',
                      'G16PREIFUE','G16PRELJOH',
                      'G16PREGSTE','G16PRERTRU',
                      'G16PREOWRI','G16HALDLIN',
                      'G16HALLMCD',
                      'G16HALISOU',
                      'G16HALOWRI','G16HALRYOU',
                      'G16USSICRA','G16USSIGIA',
                      'G16USSDMET','G16USSLMIL',
                      'G16USSRMUR','G16USSISTO',
                      'G16USSOWRI']

ak_2016_pvt.columns = data_columns
ak_2016_pvt = ak_2016_pvt[['G16PRERTRU', 'G16PREDCLI', 'G16PRELJOH',
       'G16PREGSTE', 'G16PRECCAS', 'G16PREIFUE', 'G16PREOWRI', 'G16USSRMUR',
       'G16USSDMET', 'G16USSLMIL', 'G16USSICRA', 'G16USSIGIA', 'G16USSISTO',
       'G16USSOWRI', 'G16HALRYOU', 'G16HALDLIN', 'G16HALLMCD', 'G16HALISOU',
       'G16HALOWRI']]

Index(['US PRESIDENT -Castle, Darrell L. ', 'US PRESIDENT -Clinton, Hillary ',
       'US PRESIDENT -De La Fuente, Roque ', 'US PRESIDENT -Johnson, Gary ',
       'US PRESIDENT -Stein, Jill ', 'US PRESIDENT -Trump, Donald J. ',
       'US PRESIDENT -Write-in 60 ', 'US REPRESENTATIVE -Lindbeck, Steve ',
       'US REPRESENTATIVE -McDermott, Jim C.  ',
       'US REPRESENTATIVE -Souphanavong, Bernie ',
       'US REPRESENTATIVE -Write-in 50 ', 'US REPRESENTATIVE -Young, Don ',
       'US SENATOR -Craig, Breck A.  ', 'US SENATOR -Gianoutsos, Ted ',
       'US SENATOR -Metcalfe, Ray ', 'US SENATOR -Miller, Joe ',
       'US SENATOR -Murkowski, Lisa ', 'US SENATOR -Stock, Margaret ',
       'US SENATOR -Write-in 70 '],
      dtype='object', name='race_cand')


## Check election results at the race total level

In [11]:
print("Printing differences below:")
for race in data_columns:
    if (sum(vest_ak_16[race])-sum(ak_2016_pvt[race]) != 0):
        print(race+" has a difference of "+str(sum(vest_ak_16[race])-sum(ak_2016_pvt[race]))+" votes")
        print("\tVEST: "+str(sum(vest_ak_16[race]))+" votes")
        print("\tSOURCES: "+str(sum(ak_2016_pvt[race]))+" votes")
print("")
print("All other races are equal")

Printing differences below:

All other races are equal


## Allocate district wide and statewide votes

### Setup

From VEST's documentation:

> Early, Absentee, and Questioned votes are only reported at the State House district level (since Alaska has portions of the state with no lower level of government, HDs serve a similar purpose as counties do in other states for the purpose of reporting votes). These votes are apportioned to precincts by candidate in the same shares that the Election Day vote was split among precincts within an HD.

#### Make the current index column a column in the df

In [12]:
ak_2016_pvt.reset_index(drop=False,inplace=True)

#### Group the votes that need to be allocated together

Note: There are two categories here:
1. The various votes recorded at the State House district level that VEST describes in their documentation
2. The "HD99 Fed Overseas Absentee " votes that VEST does not mention, but mentions in their VEST AK 18 documentation

From VEST's AK 18 documentation: 

>Similarly, federal-only ballots that are reported at the statewide level were apportioned to precincts by candidate based on their share of the precinct-level vote.

#### Create a list of the State House district reported votes

In [13]:
to_allocate = list(ak_2016_pvt[ak_2016_pvt[0].str.contains("District")][0])


#### Create the three needed dataframes: State House district votes, precinct-level votes and the one statewide votes

In [14]:
ak_2016_to_allocate = ak_2016_pvt[ak_2016_pvt[0].isin(to_allocate)]
ak_2016_precinct = ak_2016_pvt[~(ak_2016_pvt[0].isin(to_allocate)) & (ak_2016_pvt[0]!="HD99 Fed Overseas Absentee ")]
ak_2016_fed_overseas = ak_2016_pvt[ak_2016_pvt[0]=="HD99 Fed Overseas Absentee "]



#### Check the shapes to make sure everything accounted for and not counted twice

In [15]:
print(ak_2016_to_allocate.shape)
print(ak_2016_precinct.shape)
print(ak_2016_fed_overseas.shape)
print(ak_2016_pvt.shape)

(120, 20)
(441, 20)
(1, 20)
(562, 20)


### Allocate the state house district votes

#### Create a two digit "District" column in the precinct-level votes and the state house district votes

In [16]:
ak_2016_precinct.loc[:,"District"] = ak_2016_precinct.loc[:,0].apply(lambda x: x[0:2])
ak_2016_to_allocate.loc[:,"col_names"]=ak_2016_to_allocate.loc[:,0].str.split(" - ")
ak_2016_to_allocate.loc[:,"District"]=ak_2016_to_allocate.loc[:,"col_names"].apply(lambda x: x[0][9:].zfill(2))

/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, pi)


#### Get the totals of how many votes need to be allocated per State House district

In [17]:
to_dole_out_totals = pd.DataFrame(ak_2016_to_allocate.groupby(["District"]).sum())
to_dole_out_totals.reset_index(inplace=True,drop=False)



#### Get the totals precinct-reported votes in each State House district

In [18]:
precinct_specific_totals = pd.DataFrame(ak_2016_precinct.groupby(["District"]).sum())
precinct_specific_totals.reset_index(inplace=True,drop=False)



#### Take a look at these two

In [19]:
print(precinct_specific_totals)
print(to_dole_out_totals)



   District  G16PRERTRU  G16PREDCLI  G16PRELJOH  G16PREGSTE  G16PRECCAS  \
0        01        1965        1392         270          94          46   
1        02        1837         760         199          52          42   
2        03        3620         751         313          63          83   
3        04        2714        2382         373         154          71   
4        05        2373        1823         313         112          58   
5        06        3198        1558         284          71          98   
6        07        3143         717         231          72          58   
7        08        3845         666         240          64          81   
8        09        3853        1092         331          84          91   
9        10        3657         947         296          95          79   
10       11        3866        1145         367          86          48   
11       12        3989        1049         350          64          60   
12       13        2413  

#### Run the function to allocate the votes

In [20]:
races=['G16PRECCAS','G16PREDCLI','G16PREIFUE','G16PRELJOH',
    'G16PREGSTE','G16PRERTRU','G16PREOWRI','G16HALDLIN','G16HALLMCD',
    'G16HALISOU','G16HALOWRI','G16HALRYOU','G16USSICRA','G16USSIGIA',
    'G16USSDMET','G16USSLMIL','G16USSRMUR','G16USSISTO','G16USSOWRI']

#Create some new columns for each of these races to deal with the allocation
for race in races:
    add_var = race+"_add"
    rem_var = race+"_rem"
    floor_var = race+"_floor"
    ak_2016_precinct.loc[:,add_var]=0.0
    ak_2016_precinct.loc[:,rem_var]=0.0
    ak_2016_precinct.loc[:,floor_var]=0.0

#Iterate over the rows
#Note this function iterates over the dataframe two times so the rounded vote totals match the totals to allocate
for index, row in ak_2016_precinct.iterrows():
    for race in races:
        add_var = race+"_add"
        rem_var = race+"_rem"
        floor_var = race+"_floor"
        #Grab the district
        county_id = row["District"]
        #Get the denominator for the allocation (the precinct vote totals)
        denom = precinct_specific_totals.loc[precinct_specific_totals["District"]==county_id][race]
        #Get one of the numerators, how many districtwide votes to allocate
        numer = to_dole_out_totals.loc[to_dole_out_totals["District"]==county_id][race]
        #Get the vote totals for this race in this precinct
        val = ak_2016_precinct.at[index,race]
        #Get the vote share, the precincts % of total precinct votes in the district times votes to allocate
        vote_share = (float(val)/float(denom))*float(numer)
        ak_2016_precinct.at[index,add_var] = vote_share
        #Take the decimal remainder of the allocation
        ak_2016_precinct.at[index,rem_var] = vote_share%1
        #Take the floor of the allocation
        ak_2016_precinct.at[index,floor_var] = np.floor(vote_share)

#After the first pass through, get the sums of the races by district to assist in the rounding            
first_allocation = pd.DataFrame(ak_2016_precinct.groupby(["District"]).sum())

#Now we want to iterate district by district to work on rounding
county_list = list(to_dole_out_totals["District"].unique()) 

#Iterate over the district
for county in county_list:
    for race in races:
        add_var = race+"_add"
        rem_var = race+"_rem"
        floor_var = race+"_floor"
        #County how many votes still need to be allocated (because we took the floor of all the initial allocations)
        to_go = int(np.round((int(to_dole_out_totals.loc[to_dole_out_totals["District"]==county][race])-first_allocation.loc[first_allocation.index==county,floor_var])))
        #Grab the n precincts with the highest remainders and round these up, where n is the # of votes that still need to be allocated
        for index in ak_2016_precinct.loc[ak_2016_precinct["District"]==county][rem_var].nlargest(to_go).index:
            ak_2016_precinct.at[index,add_var] = np.ceil(ak_2016_precinct.at[index,add_var])

#Iterate over every race again
for race in races:
    add_var = race+"_add"
    #Round every allocation down to not add fractional votes
    ak_2016_precinct.loc[:,add_var]=np.floor(ak_2016_precinct.loc[:,add_var])

/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying

Note that after running the above, there is a column for every race with "_add" appended to it, that has the number of votes that need to be added as part of this allocation. We will hold off on actually adding these votes until the allocation of the statewide votes, so as not to affect that.

### Allocate the statewide votes

#### Get the precinct-level vote totals (not broken out by State House district)

In [21]:
totals = pd.DataFrame(ak_2016_precinct.sum())
precinct_vote_total_across_districts = totals.T

#### Take a look at the totals and the statewide votes

In [22]:
print(precinct_vote_total_across_districts)
print(ak_2016_fed_overseas)

                                                   0 G16PRERTRU G16PREDCLI  \
0  01-446 Aurora 01-455 Fairbanks No. 1 01-465 Fa...     103457      69097   

  G16PRELJOH G16PREGSTE G16PRECCAS G16PREIFUE G16PREOWRI G16USSRMUR  \
0      12004       3782       2879        835       5870      87854   

  G16USSDMET  ... G16USSLMIL_floor G16USSRMUR_add G16USSRMUR_rem  \
0      19602  ...          29831.0        50224.0          222.0   

  G16USSRMUR_floor G16USSISTO_add G16USSISTO_rem G16USSISTO_floor  \
0          50002.0        15818.0          214.0          15604.0   

  G16USSOWRI_add G16USSOWRI_rem G16USSOWRI_floor  
0          283.0           90.0            193.0  

[1 rows x 78 columns]
                               0  G16PRERTRU  G16PREDCLI  G16PRELJOH  \
561  HD99 Fed Overseas Absentee           40         274           7   

     G16PREGSTE  G16PRECCAS  G16PREIFUE  G16PREOWRI  G16USSRMUR  G16USSDMET  \
561          11           0           1           9          71         193

#### Run the function to allocate the votes

In [23]:
races=['G16PRECCAS','G16PREDCLI','G16PREIFUE','G16PRELJOH',
    'G16PREGSTE','G16PRERTRU','G16PREOWRI','G16HALDLIN','G16HALLMCD',
    'G16HALISOU','G16HALOWRI','G16HALRYOU','G16USSICRA','G16USSIGIA',
    'G16USSDMET','G16USSLMIL','G16USSRMUR','G16USSISTO','G16USSOWRI']

#Create some new columns for each of these races to deal with the allocation
for race in races:
    add_var = race+"_fed_add"
    rem_var = race+"_fed_rem"
    floor_var = race+"_fed_floor"
    ak_2016_precinct.loc[:,add_var]=0.0
    ak_2016_precinct.loc[:,rem_var]=0.0
    ak_2016_precinct.loc[:,floor_var]=0.0

for index, row in ak_2016_precinct.iterrows():
    for race in races:
            add_var = race+"_fed_add"
            rem_var = race+"_fed_rem"
            floor_var = race+"_fed_floor"
            #Get the denominator for the allocation (the precinct vote totals)
            denom = precinct_vote_total_across_districts.iloc[0][race]
            #Get one of the numerators, how many countywide votes to allocate
            numer = ak_2016_fed_overseas.iloc[0][race]
            #Get the vote totals for this race in this precinct
            val = ak_2016_precinct.at[index,race]
            #Get the vote share, the precincts % of total precinct votes in the county times votes to allocate
            vote_share = (float(val)/float(denom))*float(numer)
            ak_2016_precinct.at[index,add_var] = vote_share
            #Take the decimal remainder of the allocation
            ak_2016_precinct.at[index,rem_var] = vote_share%1
            #Take the floor of the allocation
            ak_2016_precinct.at[index,floor_var] = np.floor(vote_share)
            
#After the first pass through, get the sums of the races by county 
totals = pd.DataFrame(ak_2016_precinct.sum())
first_allocation = totals.T

 
for race in races:
    add_var = race+"_fed_add"
    rem_var = race+"_fed_rem"
    floor_var = race+"_fed_floor"
    #County how many votes still need to be allocated (because we took the floor of all the initial allocations)
    to_go = int(np.round((int(ak_2016_fed_overseas.iloc[0][race])-first_allocation.iloc[0][floor_var])))
    #Grab the n precincts with the highest remainders and round these up, where n is the # of votes that still need to be allocated
    for index in ak_2016_precinct[rem_var].nlargest(to_go).index:
        ak_2016_precinct.at[index,add_var] = np.ceil(ak_2016_precinct.at[index,add_var])

#Iterate over every race again
for race in races:
    add_var = race+"_fed_add"
    #Round every allocation down
    ak_2016_precinct.loc[:,add_var]=np.floor(ak_2016_precinct.loc[:,add_var])

/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)
/Users/peterhorton/opt/anaconda3/envs/geo_env/lib/python3.9/site-packages/pandas/core/indexing.py:1676: SettingWithCopyWarning: 
A value is trying

Note that after running the above, there is a column for every race with "_fed_add" appended to it, that has the number of votes that need to be added as part of this allocation. We will hold off on actually adding these votes until the allocation of the statewide votes, so as not to affect that.

In [24]:
#### Quick check to make sure that we didn't add too many votes

In [25]:
print("Printing issues below: ")
for race in races:
    add_var = race+"_fed_add"
    add_var_2 = race+"_add"
    diff_1 = int(ak_2016_fed_overseas.iloc[0][race]-ak_2016_precinct.loc[:,add_var].sum())
    diff_2 = int(to_dole_out_totals.loc[:,race].sum()-ak_2016_precinct.loc[:,add_var_2].sum())
    if((diff_1 != 0) & (diff_2 !=0)):
        print(diff_1)
        print(diff_2)
        print("Issue with "+ race)
print("")
print("All others match")

Printing issues below: 

All others match


### Add the State House District and Statewide allocated votes and filter down to original columns

In [26]:
for race in races:
    add_var = race+"_fed_add"
    add_var_2 = race+"_add"
    ak_2016_precinct.loc[:,race]+=ak_2016_precinct.loc[:,add_var]
    ak_2016_precinct.loc[:,race]+=ak_2016_precinct.loc[:,add_var_2]
    
ak_2016_precinct = ak_2016_precinct[[                 0,       'G16PRERTRU',       'G16PREDCLI',
             'G16PRELJOH',       'G16PREGSTE',       'G16PRECCAS',
             'G16PREIFUE',       'G16PREOWRI',       'G16USSRMUR',
             'G16USSDMET',       'G16USSLMIL',       'G16USSICRA',
             'G16USSIGIA',       'G16USSISTO',       'G16USSOWRI',
             'G16HALRYOU',       'G16HALDLIN',       'G16HALLMCD',
             'G16HALISOU',       'G16HALOWRI',         'District']]

In [27]:
#### Check the totals again just to be sure

In [28]:
print("Printing differences below:")
for race in data_columns:
    if (sum(vest_ak_16[race])-sum(ak_2016_precinct[race]) != 0):
        print(race+" has a difference of "+str(sum(vest_ak_16[race])-sum(ak_2016_precinct[race]))+" votes")
        print("\tVEST: "+str(sum(vest_ak_16[race]))+" votes")
        print("\tSOURCES: "+str(sum(ak_2016_precinct[race]))+" votes")
print("")
print("All other races are equal")

Printing differences below:

All other races are equal


### Check vote totals by district

In [29]:
print(ak_2016_precinct["District"].unique())
vest_ak_16["District_mod"] = vest_ak_16["DISTRICT"].str[0:2]
print(vest_ak_16["District_mod"].unique())

['01' '02' '03' '04' '05' '06' '07' '08' '09' '10' '11' '12' '13' '14'
 '15' '16' '17' '18' '19' '20' '21' '22' '23' '24' '25' '26' '27' '28'
 '29' '30' '31' '32' '33' '34' '35' '36' '37' '38' '39' '40']
['11' '12' '10' '15' '28' '16' '17' '18' '19' '20' '21' '22' '23' '24'
 '25' '26' '27' '13' '14' '07' '08' '29' '30' '31' '32' '33' '34' '35'
 '36' '01' '02' '03' '04' '05' '06' '09' '37' '38' '39' '40']


In [30]:
diff_counties=[]
for i in data_columns:
    diff = ak_2016_precinct.groupby(["District"]).sum()[i]-vest_ak_16.groupby(["District_mod"]).sum()[i]
    for val in diff[diff != 0].index.values.tolist():
        if val not in diff_counties:
            diff_counties.append(val)
    if len(diff[diff != 0]!=0):
        print(i)
        print(diff[diff != 0].to_string(header=False))
print("All other races in all counties are equal")

G16PREDCLI
02   -1.0
07   -1.0
10   -1.0
11   -1.0
18   -1.0
19    1.0
25    1.0
27    1.0
29   -1.0
30   -3.0
31   -2.0
33   -1.0
34   -3.0
37    3.0
38    2.0
39    5.0
40    2.0
G16PREIFUE
34   -1.0
35    1.0
G16PRELJOH
04    1.0
13   -1.0
G16PREGSTE
28    1.0
32   -1.0
G16PRERTRU
02   -1.0
11   -1.0
14    1.0
24    1.0
G16PREOWRI
11   -1.0
40    1.0
G16HALDLIN
07   -2.0
08    1.0
09    1.0
11    1.0
12   -1.0
18   -2.0
20   -1.0
23    2.0
25    1.0
26    1.0
27    1.0
30   -1.0
32    1.0
33   -3.0
34   -1.0
36    1.0
39    1.0
G16HALLMCD
04   -1.0
29    1.0
G16HALISOU
03    1.0
31   -1.0
G16HALOWRI
07    1.0
10   -1.0
G16HALRYOU
14   -1.0
22    1.0
24    1.0
27    2.0
29   -2.0
34   -1.0
G16USSDMET
01   -1.0
07   -1.0
09    1.0
10   -1.0
11   -1.0
13   -1.0
19    1.0
23    1.0
31   -1.0
33   -1.0
34   -1.0
36    2.0
38    1.0
39    1.0
40    1.0
G16USSLMIL
07   -1.0
09    1.0
G16USSRMUR
07   -1.0
08   -2.0
12   -1.0
15   -1.0
23    1.0
25    1.0
26    1.0
27    2.0
28    1.0
29   -

Note: Although there are some differences, the largest difference is 5 and this was to be expected given possible different implementations of the vote allocation method.

#### Rename the first column, to be used in the shapefile join

In [31]:
ak_2016_precinct = ak_2016_precinct.rename(columns={0: 'NAME'})


In [32]:
print(ak_2016_precinct.shape)

(441, 21)


## Shapefile

### Load the shapefile, take a look

In [33]:
ak_shp = gp.read_file("./raw-from-source/Shapefiles/SW Proc Shape Files/2013-SW-Proc-Shape-files.shp")
print(ak_shp)

      ID          AREA DISTRICT  MEMBERS LOCKED                       NAME  \
0    173      1.565101   11-075      1.0   None   11-075 Palmer City No. 2   
1    181    286.270233   12-220      1.0   None               12-220 Butte   
2    182      3.675227   10-055      1.0   None         10-055 Lakes No. 2   
3     32     29.413515   15-300      1.0   None          15-300 JBER No. 2   
4    144      6.259140   28-110      1.0   None  28-110 Rabbit Creek No. 2   
..   ...           ...      ...      ...    ...                        ...   
436  461  10066.747000   40-034      1.0   None         40-034 POINT LAY\n   
437  462   4899.020990   40-036      1.0   None           40-036 SELAWIK\n   
438  463   1661.107910   40-038      1.0   None          40-038 SHUNGNAK\n   
439  464   9258.007810   40-040      1.0   None        40-040 WAINWRIGHT\n   
440  465      2.990466   40-330      1.0   None            40-330 HUGHES\n   

     POPULATION  IDEAL_VALU  DEVIATION  F_DEVIATIO  \
0        

### Look at the shape, if 441 good to try to join

In [34]:
print(ak_shp.shape)

(441, 11)


## Shapefile and Election Result Merge

### Try to find a unique ID

In [35]:
print(ak_shp["NAME"].value_counts())
print(ak_2016_precinct["NAME"].value_counts())

19-515 South Mountain View No. 2    1
10-020 Houston                      1
23-765 Campbell Creek No. 3         1
37-304 HOLY CROSS                   1
28-115 Huffman No. 6                1
                                   ..
18-455 West Anchorage No. 3         1
39-928 NUNAM IQUA\n                 1
38-858 TUNUNAK\n                    1
28-135 Girdwood                     1
07-130 Foothills                    1
Name: NAME, Length: 441, dtype: int64
20-570 Fairview                 1
39-912 Gambell                  1
12-230 Peters Creek No. 1       1
34-410 Juneau Airport           1
39-904 Chevak                   1
                               ..
29-110 Cooper Landing           1
06-030 Eagle                    1
34-400 Auke Bay                 1
38-818 Eek                      1
23-760 Campbell Creek No. 2     1
Name: NAME, Length: 441, dtype: int64


### Attempt to join on this

In [36]:
first_join = pd.merge(ak_2016_precinct,ak_shp,how="outer",on="NAME",indicator=True)
print(first_join["_merge"].value_counts())

first_join[first_join["_merge"]=="left_only"].to_csv("./only_elections.csv")
first_join[first_join["_merge"]=="right_only"].to_csv("./only_shapes.csv")

left_only     441
right_only    441
both            0
Name: _merge, dtype: int64


### Use just the first 6 digits of name as an alternate unique ID, check if unique

In [37]:
ak_2016_precinct["join_col"]=ak_2016_precinct["NAME"].str[0:6]
ak_shp["join_col"]=ak_shp["NAME"].str[0:6]

print(ak_shp["join_col"].value_counts())
print(ak_2016_precinct["join_col"].value_counts())

26-810    1
37-300    1
08-145    1
06-024    1
37-754    1
         ..
31-380    1
23-745    1
40-020    1
33-555    1
38-830    1
Name: join_col, Length: 441, dtype: int64
26-810    1
29-170    1
08-145    1
06-024    1
37-754    1
         ..
12-200    1
23-745    1
40-020    1
33-555    1
38-830    1
Name: join_col, Length: 441, dtype: int64


### Attempt to join using this

In [38]:
first_join = pd.merge(ak_2016_precinct,ak_shp,how="outer",on="join_col",indicator=True)
print(first_join["_merge"].value_counts())

both          441
left_only       0
right_only      0
Name: _merge, dtype: int64


## Joining to VEST file

### Use VEST's district as a join column

In [39]:
vest_ak_16["join_col"]=vest_ak_16["DISTRICT"]

### Attempt to join

In [40]:
final_merge = pd.merge(vest_ak_16,first_join,how="outer",on="join_col",indicator="final_merge")
print(final_merge["final_merge"].value_counts())

both          441
left_only       0
right_only      0
Name: final_merge, dtype: int64


## Check election results precinct-by-precinct

In [41]:
def validater_row (df, column_List):
    matching_rows = 0
    different_rows = 0
    diff_list=[]
    diff_values = []
    max_diff = 0
    for j in range(0,len(df.index)):
        same = True
        for i in column_List:
            left_Data = i + "_x"
            right_Data = i + "_y"
            diff = abs(df.iloc[j][left_Data]-df.iloc[j][right_Data])
            if(diff >0):
                if(diff>0):
                    print(i, "{:.>72}".format(df.iloc[j]["join_col"]), "(V)","{:.>5}".format(int(df.iloc[j][left_Data]))," (S){:.>5}".format(int(df.iloc[j][right_Data])),"(D):{:>5}".format(int(df.iloc[j][right_Data])-int(df.iloc[j][left_Data])))           
                #print(df.iloc[j]['countypct'])
                
                diff_values.append(abs(diff))
                same = False
                if(np.isnan(diff)):
                    print("NaN value at diff is: ", df.iloc[j]["join_col"])
                    print(df.iloc[j][left_Data])
                    print(df.iloc[j][right_Data])
                if (diff>max_diff):
                    max_diff = diff
                    #print("New max diff is: ", str(max_diff))
                    #print(df.iloc[j]['cty_pct'])
        if(same != True):
            different_rows +=1
            diff_list.append(df.iloc[j]["join_col"])
        else:
            matching_rows +=1
    print("")
    print("There are ", len(df.index)," total rows")
    print(different_rows," of these rows have election result differences")
    print(matching_rows," of these rows are the same")
    print("")
    print("The max difference between any one shared column in a row is: ", max_diff)
    if(len(diff_values)!=0):
        print("The average difference is: ", str(sum(diff_values)/len(diff_values)))
    count_big_diff = len([i for i in diff_values if i > 10])
    print("There are ", str(count_big_diff), "precinct results with a difference greater than 10")
    diff_list.sort()
    print(diff_list)

In [42]:
validater_row(final_merge,races)

G16PREDCLI ..................................................................11-075 (V) ..299  (S)..298 (D):   -1
G16PREIFUE ..................................................................11-075 (V) ...10  (S)....9 (D):   -1
G16PRERTRU ..................................................................11-075 (V) ..762  (S)..763 (D):    1
G16HALDLIN ..................................................................11-075 (V) ..360  (S)..359 (D):   -1
G16HALISOU ..................................................................11-075 (V) ...51  (S)...50 (D):   -1
G16USSIGIA ..................................................................11-075 (V) ....9  (S)....8 (D):   -1
G16USSDMET ..................................................................11-075 (V) ..100  (S)...99 (D):   -1
G16USSISTO ..................................................................11-075 (V) ..121  (S)..122 (D):    1
G16PRECCAS ..................................................................12-220 (V) 

G16PREDCLI ..................................................................24-700 (V) ..406  (S)..405 (D):   -1
G16PRELJOH ..................................................................24-700 (V) ...61  (S)...62 (D):    1
G16PREGSTE ..................................................................24-700 (V) ...28  (S)...29 (D):    1
G16HALISOU ..................................................................24-700 (V) ...30  (S)...29 (D):   -1
G16HALRYOU ..................................................................24-700 (V) ..642  (S)..643 (D):    1
G16USSDMET ..................................................................24-700 (V) ..141  (S)..142 (D):    1
G16USSRMUR ..................................................................24-700 (V) ..513  (S)..514 (D):    1
G16PREDCLI ..................................................................25-840 (V) ..159  (S)..160 (D):    1
G16PRERTRU ..................................................................25-840 (V) 

G16PRELJOH ..................................................................13-235 (V) ...50  (S)...49 (D):   -1
G16PREOWRI ..................................................................13-235 (V) ...32  (S)...33 (D):    1
G16HALISOU ..................................................................13-235 (V) ...29  (S)...28 (D):   -1
G16USSIGIA ..................................................................13-235 (V) ...12  (S)...13 (D):    1
G16USSLMIL ..................................................................13-235 (V) ..328  (S)..327 (D):   -1
G16USSISTO ..................................................................13-235 (V) ...73  (S)...74 (D):    1
G16PRELJOH ..................................................................12-233 (V) ...49  (S)...48 (D):   -1
G16PREOWRI ..................................................................12-233 (V) ...20  (S)...21 (D):    1
G16HALDLIN ..................................................................12-233 (V) 

G16HALLMCD ..................................................................16-340 (V) ...69  (S)...68 (D):   -1
G16HALRYOU ..................................................................16-340 (V) ..429  (S)..430 (D):    1
G16USSLMIL ..................................................................16-340 (V) ..228  (S)..227 (D):   -1
G16USSRMUR ..................................................................16-340 (V) ..416  (S)..417 (D):    1
G16PRECCAS ..................................................................16-350 (V) ....8  (S)....7 (D):   -1
G16PREDCLI ..................................................................16-350 (V) ..710  (S)..711 (D):    1
G16PREIFUE ..................................................................16-350 (V) ...11  (S)...10 (D):   -1
G16PRERTRU ..................................................................16-350 (V) ..784  (S)..785 (D):    1
G16PREOWRI ..................................................................16-350 (V) 

G16HALRYOU ..................................................................18-445 (V) ..301  (S)..302 (D):    1
G16USSIGIA ..................................................................18-445 (V) ....2  (S)....3 (D):    1
G16USSRMUR ..................................................................18-445 (V) ..318  (S)..317 (D):   -1
G16USSISTO ..................................................................18-445 (V) ..166  (S)..167 (D):    1
G16USSOWRI ..................................................................18-445 (V) ....6  (S)....5 (D):   -1
G16PRECCAS ..................................................................18-450 (V) ....7  (S)....5 (D):   -2
G16PREDCLI ..................................................................18-450 (V) ..334  (S)..335 (D):    1
G16PREIFUE ..................................................................18-450 (V) ....4  (S)....3 (D):   -1
G16PRELJOH ..................................................................18-450 (V) 

...

G16PREDCLI ..................................................................38-808 (V) ..396  (S)..395 (D):   -1
G16PRELJOH ..................................................................38-808 (V) ...42  (S)...41 (D):   -1
G16PRERTRU ..................................................................38-808 (V) ..234  (S)..235 (D):    1
G16PREOWRI ..................................................................38-808 (V) ...43  (S)...42 (D):   -1
G16HALRYOU ..................................................................38-808 (V) ..329  (S)..330 (D):    1
G16USSIGIA ..................................................................38-808 (V) ....5  (S)....4 (D):   -1
G16USSDMET ..................................................................38-808 (V) ...74  (S)...76 (D):    2
G16USSISTO ..................................................................38-808 (V) ..108  (S)..107 (D):   -1
G16PRECCAS ..................................................................38-810 (V) 

G16PREGSTE ..................................................................38-846 (V) ....3  (S)....4 (D):    1
G16PRERTRU ..................................................................38-846 (V) ...26  (S)...27 (D):    1
G16PREOWRI ..................................................................38-846 (V) ....3  (S)....4 (D):    1
G16HALDLIN ..................................................................38-846 (V) ...46  (S)...47 (D):    1
G16HALISOU ..................................................................38-846 (V) ....5  (S)....6 (D):    1
G16USSLMIL ..................................................................38-846 (V) ...29  (S)...30 (D):    1
G16USSRMUR ..................................................................38-846 (V) ...69  (S)...68 (D):   -1
G16PRECCAS ..................................................................38-848 (V) ...12  (S)...11 (D):   -1
G16PREDCLI ..................................................................38-848 (V) 

G16HALLMCD ..................................................................39-910 (V) ....3  (S)....4 (D):    1
G16HALISOU ..................................................................39-910 (V) ....5  (S)....6 (D):    1
G16USSLMIL ..................................................................39-910 (V) ...36  (S)...37 (D):    1
G16PREDCLI ..................................................................39-912 (V) ..147  (S)..148 (D):    1
G16PRELJOH ..................................................................39-912 (V) ...13  (S)...12 (D):   -1
G16PREGSTE ..................................................................39-912 (V) ....3  (S)....4 (D):    1
G16PRERTRU ..................................................................39-912 (V) ...32  (S)...31 (D):   -1
G16PREOWRI ..................................................................39-912 (V) ....5  (S)....6 (D):    1
G16HALLMCD ..................................................................39-912 (V) 

G16PRELJOH ..................................................................39-952 (V) ....4  (S)....5 (D):    1
G16PREOWRI ..................................................................39-952 (V) ...13  (S)...12 (D):   -1
G16HALLMCD ..................................................................39-952 (V) ....2  (S)....3 (D):    1
G16USSLMIL ..................................................................39-952 (V) ...15  (S)...14 (D):   -1
G16PREIFUE ..................................................................40-002 (V) ....2  (S)....3 (D):    1
G16PRELJOH ..................................................................40-002 (V) ....7  (S)....6 (D):   -1
G16HALLMCD ..................................................................40-002 (V) ....9  (S)....8 (D):   -1
G16USSLMIL ..................................................................40-002 (V) ...23  (S)...24 (D):    1
G16USSRMUR ..................................................................40-002 (V) 

G16PRERTRU ..................................................................40-034 (V) ...15  (S)...16 (D):    1
G16HALDLIN ..................................................................40-034 (V) ...22  (S)...23 (D):    1
G16HALLMCD ..................................................................40-034 (V) ....4  (S)....5 (D):    1
G16HALRYOU ..................................................................40-034 (V) ...49  (S)...48 (D):   -1
G16PRECCAS ..................................................................40-036 (V) ....4  (S)....5 (D):    1
G16PRELJOH ..................................................................40-036 (V) ....7  (S)....6 (D):   -1
G16PRERTRU ..................................................................40-036 (V) ...44  (S)...43 (D):   -1
G16PREOWRI ..................................................................40-036 (V) ....8  (S)....7 (D):   -1
G16USSDMET ..................................................................40-036 (V) 

Note: If you scroll down in the above block, you can read the following:
        
>There are  441  total rows  
440  of these rows have election result differences  
1  of these rows are the same

> The max difference between any one shared column in a row is:  2.0  
The average difference is:  1.0304761904761905  
There are  0 precinct results with a difference greater than 10  

## Check geometries

In [43]:
both = final_merge[final_merge["final_merge"]=="both"]
vest_geoms = gp.GeoDataFrame(both,geometry="geometry_x",crs=vest_ak_16.crs)
source_geoms = gp.GeoDataFrame(both,geometry="geometry_y",crs=vest_ak_16.crs)
source_geoms = source_geoms.to_crs(3857)
vest_geoms = vest_geoms.to_crs(3857)
source_geoms["geometry_x"]=source_geoms.buffer(0)
vest_geoms["geometry_y"]=vest_geoms.buffer(0)
vals = source_geoms.geom_almost_equals(vest_geoms,decimal=0)
print(vals.value_counts())

True    441
dtype: int64


In [44]:
count = 0
area_list = []
for i in range(0,len(source_geoms)):
    diff = source_geoms.iloc[[i]].symmetric_difference(vest_geoms.iloc[[i]])
    intersection = source_geoms.iloc[[i]].intersection(vest_geoms.iloc[[i]])
    area = float(diff.area/10e6)
    area_list.append(area)
    #print("Area is " + str(area))

    if (area > .1):
        count += 1
        name = source_geoms.iat[i,2]
        
        print(str(count)+") For " + name + " difference in area is " + str(area))
        if (intersection.iloc[0].is_empty):
            base = diff.plot(color="red")
            source_geoms.iloc[[i]].plot(color="orange",ax=base)
            vest_geoms.iloc[[i]].plot(color="blue",ax=base)
            base.set_title(name)
        else:
            base = diff.plot(color="red")
            source_geoms.iloc[[i]].plot(color="orange",ax=base)
            vest_geoms.iloc[[i]].plot(color="blue",ax=base)
            intersection.plot(color="green",ax=base)
            base.set_title(name)

In [45]:
df = pd.DataFrame(area_list)
print(df.shape)

print(str(len(df[df[0]==0]))+" precincts w/ a difference of 0 km^2")
print(str(len(df[(df[0]<.1) & (df[0]>0)]))+ " precincts w/ a difference between 0 and .1 km^2")
print(str(len(df[(df[0]<.5) & (df[0]>=.1)]))+ " precincts w/ a difference between .1 and .5 km^2")
print(str(len(df[(df[0]<1) & (df[0]>=.5)]))+ " precincts w/ a difference between .5 and 1 km^2")
print(str(len(df[(df[0]<2) & (df[0]>=1)]))+ " precincts w/ a difference between 1 and 2 km^2")
print(str(len(df[(df[0]<5) & (df[0]>=2)]))+ " precincts w/ a difference between 2 and 5 km^2")
print(str(len(df[(df[0]>=5)]))+ " precincts w/ a difference greater than 5 km^2")

(441, 1)
441 precincts w/ a difference of 0 km^2
0 precincts w/ a difference between 0 and .1 km^2
0 precincts w/ a difference between .1 and .5 km^2
0 precincts w/ a difference between .5 and 1 km^2
0 precincts w/ a difference between 1 and 2 km^2
0 precincts w/ a difference between 2 and 5 km^2
0 precincts w/ a difference greater than 5 km^2
